In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

# Carregar os dados
df = pd.read_csv('marketing_campaign.csv', sep="\t")

# Pré-processamento
df = df.drop(columns=['Dt_Customer'])
df = df.dropna(subset=['Income'])
df.reset_index(drop=True, inplace=True)

# Definir colunas
numericas_continuas = [
    'Year_Birth', 'Income', 'Recency', 'MntWines', 'MntFruits',
    'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
    'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
    'NumStorePurchases', 'NumWebVisitsMonth'
]

numericas_binarias = [
    'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
    'AcceptedCmp2', 'Complain', 'Response'
]

categoricas = ['Education', 'Marital_Status']

# Codificar variáveis categóricas
education_order = {'Basic': 0, '2n Cycle': 1, 'Graduation': 2, 'Master': 3, 'PhD': 4}
df['Education_encoded'] = df['Education'].map(education_order)
df = pd.get_dummies(df, columns=['Marital_Status'], prefix='Marital', drop_first=False)

# Criar DataFrame para DBSCAN (sem ID e colunas não utilizadas)
df_dbscan = df.drop(['ID', 'Education', 'Z_CostContact', 'Z_Revenue'], axis=1).copy()

# Aplicar MinMaxScaler APENAS nas variáveis contínuas
scaler = MinMaxScaler()
df_dbscan[numericas_continuas] = scaler.fit_transform(df_dbscan[numericas_continuas])

# Otimização de parâmetros
melhor_score = -1
melhor_eps = None
melhor_min_samples = None

for eps in np.arange(0.5, 5, 0.1):  # Range mais concentrado
    for min_samples in range(3, 11):   # Testando mais valores
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(df_dbscan)
        
        # Ignorar se todos os pontos forem ruído ou um único cluster
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if n_clusters >= 2:
            try:
                score = silhouette_score(df_dbscan, labels)
                if score > melhor_score:
                    melhor_score = score
                    melhor_eps = eps
                    melhor_min_samples = min_samples
            except:
                continue

print(f"\nMelhores parâmetros: eps={melhor_eps}, min_samples={melhor_min_samples}")
print(f"Melhor silhouette score: {melhor_score:.4f}")

# Aplicar DBSCAN com os melhores parâmetros
dbscan = DBSCAN(eps=melhor_eps, min_samples=melhor_min_samples)
df_dbscan['cluster_dbscan'] = dbscan.fit_predict(df_dbscan)

# Análise dos resultados
n_clusters = df_dbscan['cluster_dbscan'].nunique() - (1 if -1 in df_dbscan['cluster_dbscan'].values else 0)
print(f'\nNúmero de clusters: {n_clusters}')
print(df_dbscan['cluster_dbscan'].value_counts().sort_index())

# Verificar proporção de ruído
ruido_prop = (df_dbscan['cluster_dbscan'] == -1).mean()
print(f'\nProporção de ruído: {ruido_prop:.2%}')


Melhores parâmetros: eps=0.9999999999999999, min_samples=3
Melhor silhouette score: 0.2441

Número de clusters: 135
cluster_dbscan
-1      450
 0       13
 1       23
 2       37
 3       55
       ... 
 130      5
 131      3
 132      3
 133      4
 134      3
Name: count, Length: 136, dtype: int64

Proporção de ruído: 20.31%
